In [7]:
########## 2回目
import pandas as pd
import numpy as np
import sys
sys.path.append("../_utils") # システムパスに対象ディレクトリを追加
import utils

train = pd.read_csv('./data/train.csv', index_col=0)
test = pd.read_csv('./data/test.csv', index_col=0)
sample_submit = pd.read_csv('./data/sample_submission.csv', index_col=0, header=None)

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

# 訓練データとテストデータに分割
# stratifyを設定して目的変数の分布を保ったまま分割
train, valid = train_test_split(train, test_size=0.2, stratify=train['health'], random_state=82)

# カテゴリカルな特徴量を数値に変換する
categorical_cols = ['curb_loc','steward','guards','sidewalk','user_type','problems','spc_common','spc_latin','nta','nta_name','boroname','zip_city']

train = utils.bulk_label_encoding(train, categorical_cols)
test = utils.bulk_label_encoding(test, categorical_cols)
# print(test)

# 使用する特徴量の選択
target_cols =['guards','problems','steward','spc_common','sidewalk','user_type','spc_latin']

# 目的変数とそれ以外に学習用データを分割
x_train = train[target_cols]
y_train = train['health']
x_valid = valid[target_cols]
y_valid = valid['health']
x_test = test[target_cols]

# モデルを作成して訓練
model = RandomForestClassifier()
model.fit(x_train, y_train)
print(x_valid)
# validの特徴量で予測
valid_predictions = model.predict(x_valid)
valid_f1 = f1_score(y_valid, valid_predictions, average='macro')
print(f"Validation F1 Score (Macro): {valid_f1}")

# 予測結果をCSVファイルとして保存
pred = model.predict(x_test)
sample_submit[1] = pred
sample_submit.to_csv('./submit/submit2.csv', header=None)

        guards             problems steward           spc_common  sidewalk  \
17481  Helpful                  NaN    3or4               cherry    Damage   
19264  Helpful                  NaN    1or2              Sophora    Damage   
5000       NaN  RootOtherTrunkOther     NaN   crimson king maple    Damage   
8185       NaN                  NaN     NaN     London planetree  NoDamage   
10579      NaN                  NaN     NaN    European hornbeam    Damage   
...        ...                  ...     ...                  ...       ...   
4565       NaN               Stones     NaN            red maple    Damage   
1742       NaN         BranchLights     NaN  Kentucky coffeetree    Damage   
37         NaN                  NaN    1or2   crimson king maple  NoDamage   
14201  Helpful                  NaN    3or4         Callery pear    Damage   
15441      NaN                  NaN     NaN      American linden    Damage   

              user_type                        spc_latin  
1748

ValueError: could not convert string to float: 'Helpful'